In [1]:
from funcs import *
import pandas as pd

In [2]:
read_all('ARCSIX-LARGE-OPTICAL_P3B')

,Time_Mid,Sc450_total,Sc550_total,Sc700_total,Sc450_submicron,Sc550_submicron,Sc700_submicron,Sc550_submicron_amb,Abs470_total,Abs532_total,...,gamma550,fRH550_RH20to80,AEscatDRY_450to700nm,AEscatAMB_450to700nm,AEabsDRY_470to660nm,SSA_dry_450nm,SSA_dry_550nm,SSA_dry_700nm,SSA_amb_550nm,stdPT
0,2024-05-28 11:58:07,-0.75,-2.30,-2.33,2.86,2.04,1.44,NaN,NaN,NaN,...,NaN,NaN,2.03,1.61,NaN,NaN,NaN,NaN,NaN,1.005
1,2024-05-28 11:58:08,-1.00,-1.35,1.73,3.18,1.96,1.73,NaN,NaN,NaN,...,NaN,NaN,2.00,1.57,NaN,NaN,NaN,NaN,NaN,1.004
2,2024-05-28 11:58:09,1.75,-0.55,6.60,3.49,1.88,2.03,NaN,NaN,NaN,...,NaN,NaN,1.99,1.55,NaN,NaN,NaN,NaN,NaN,1.005
3,2024-05-28 11:58:10,1.14,-0.83,-3.90,3.30,1.15,2.23,NaN,NaN,NaN,...,NaN,NaN,1.97,1.56,NaN,NaN,NaN,NaN,NaN,1.006
4,2024-05-28 11:58:11,-1.65,0.23,-4.53,3.13,0.36,1.90,NaN,NaN,NaN,...,NaN,NaN,1.99,1.55,NaN,NaN,NaN,NaN,NaN,1.008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23893,2024-08-16 17:50:10,93.48,59.46,35.02,88.79,64.13,38.50,64.15,NaN,NaN,...,0.00,1.00,1.74,1.74,NaN,NaN,NaN,NaN,NaN,1.092
23894,2024-08-16 17:50:11,93.40,59.19,32.22,91.41,65.02,36.00,65.00,NaN,NaN,...,-0.00,1.00,1.73,1.73,NaN,NaN,NaN,NaN,NaN,1.093
23895,2024-08-16 17:50:12,82.70,62.22,34.03,90.52,64.97,35.15,65.01,NaN,NaN,...,0.00,1.00,1.72,1.72,NaN,NaN,NaN,NaN,NaN,1.094
23896,2024-08-16 17:50:13,82.64,63.80,35.06,89.60,64.92,34.29,64.92,NaN,NaN,...,0.00,1.00,1.72,1.72,NaN,NaN,NaN,NaN,NaN,1.094
